In [555]:
import pandas as pd
from pathlib import Path
import os

In [556]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Criando dados da tabela cabeçalho (Só é carregada uma vez, caso tenha o id da conta, não será substituido)

In [2]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\tbAccount_ig.json'
data_cabecalho = pd.read_json(file_path)
data_cabecalho.head()

,id,username,biography,followers_count,follows_count,profile_picture_url,media_count,name
0,17841425444516188,bodemeier.digital,♟️Criação de conteúdo estratégico\n▪️+ de 5 an...,2877,1388,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,90,NaN
1,17841417530400616,gabgalani,RJ 🔸 SP\ndata is the new oil 💻\n@carolinebodem...,238,374,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,139,Gabriel Galani


In [3]:
rename_column_cabecalho = {
    'id': 'id_account',
    'username': 'account_username',
    'biography': 'account_biography',
    'profile_picture_url': 'profile_picture_url',
    'name': 'account_name'
}
data_cabecalho.rename(columns=rename_column_cabecalho, inplace=True)

select_column_cabecalho = ['id_account', 'account_username', 'account_biography', 'profile_picture_url', 'account_name']
data_cabecalho = data_cabecalho[select_column_cabecalho]
data_cabecalho.head()

,id_account,account_username,account_biography,profile_picture_url,account_name
0,17841425444516188,bodemeier.digital,♟️Criação de conteúdo estratégico\n▪️+ de 5 an...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,NaN
1,17841417530400616,gabgalani,RJ 🔸 SP\ndata is the new oil 💻\n@carolinebodem...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Gabriel Galani


In [4]:
cabecalho_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCabecalho.csv'
data_cabecalho.to_csv(cabecalho_output, sep='\t')

## Criando dados da tabela de midias (criar uma chave unica, baseada na conta e data da extração)

In [65]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMedias.json'
data_midias_sem_insights = pd.read_json(file_path)
rename = {
    'id': 'id_account'
}
data_midias_sem_insights.rename(columns=rename, inplace=True)
data_midias_sem_insights.head()

,id_account,media,username
0,17841425444516188,"{'data': [{'id': '18005273918369940', 'comment...",bodemeier.digital
1,17841417530400616,"{'data': [{'id': '18014406560219838', 'comment...",gabgalani


In [66]:
df_normalized = pd.json_normalize(data_midias_sem_insights['media'])
df_normalized = pd.json_normalize(df_normalized['data'])
df = pd.concat([data_midias_sem_insights, df_normalized], axis=1)
df.drop(columns=['id_account','media','username'], inplace=True)

In [67]:
dataframes = []
for i in df.columns:
    df_json = pd.json_normalize(df[i])
    df_json = pd.concat([data_midias_sem_insights, df_json], axis=1)
    dataframes.append(df_json)

    
for i, df in enumerate(dataframes):
    dataframes[i] = df.loc[:, ~df.columns.duplicated()]
df_final = pd.concat(dataframes,ignore_index=True)
df_final.rename(columns={'id': 'id_midia'}, inplace=True)

In [68]:
select_tb_midia = ['id_account', 'username', 'id_midia', 'comments_count', 'like_count',
       'media_type', 'media_url', 'caption', 'timestamp', 'permalink',
       'media_product_type', 'thumbnail_url', 'shortcode', 'owner.id',
       'owner.username']
df_tb_midia = df_final[select_tb_midia]
df_tb_midia.head()

,id_account,username,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,shortcode,owner.id,owner.username
0,17841425444516188,bodemeier.digital,18005273918369940,2,15,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Você sabe o que é um briefing? 📝\n\n🎥 Assista ...,2024-02-20T21:19:22+0000,https://www.instagram.com/reel/C3lW1XkOcfL/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3lW1XkOcfL,17841425444516188,bodemeier.digital
1,17841417530400616,gabgalani,18014406560219838,5,21,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,Parabéns minha princesa!!!!!!!!!\n\nHoje é o s...,2024-04-03T04:01:01+0000,https://www.instagram.com/p/C5SP_m8O1S-/,FEED,NaN,C5SP_m8O1S-,17841417530400616,gabgalani
2,17841425444516188,bodemeier.digital,17846370906157882,1,13,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Uma grande dúvida que vocês me perguntam: SOCI...,2024-02-16T21:00:00+0000,https://www.instagram.com/reel/C3bDRy3uifD/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3bDRy3uifD,17841425444516188,bodemeier.digital
3,17841417530400616,gabgalani,18036730186652355,1,27,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,"Eu te amo demais meu amor, muito obrigado por ...",2024-02-26T01:51:54+0000,https://www.instagram.com/p/C3yvzhqO1Ia/,FEED,NaN,C3yvzhqO1Ia,17841417530400616,gabgalani
4,17841425444516188,bodemeier.digital,18002752616172667,0,5,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,“O maior instrumento da globalização cultural ...,2024-02-10T20:05:55+0000,https://www.instagram.com/reel/C3Lf9wLOhpf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3Lf9wLOhpf,17841425444516188,bodemeier.digital


### Tb Carrocel (Variante da Tb Midia)

In [69]:
select_carrocel = ['id_account', 'id_midia', 'media_type', 'children.data']
df_carroceu = df_final[select_carrocel]
df_carroceu = df_carroceu[df_carroceu['media_type'] == "CAROUSEL_ALBUM"]
df_carroceu.head()

,id_account,id_midia,media_type,children.data
1,17841417530400616,18014406560219838,CAROUSEL_ALBUM,"[{'media_type': 'IMAGE', 'media_url': 'https:/..."
9,17841417530400616,18012911482864852,CAROUSEL_ALBUM,"[{'media_type': 'IMAGE', 'media_url': 'https:/..."
11,17841417530400616,18015261277785871,CAROUSEL_ALBUM,"[{'media_type': 'IMAGE', 'media_url': 'https:/..."
17,17841417530400616,18316677496102466,CAROUSEL_ALBUM,"[{'media_type': 'IMAGE', 'media_url': 'https:/..."
19,17841417530400616,18049683463435392,CAROUSEL_ALBUM,"[{'media_type': 'IMAGE', 'media_url': 'https:/..."


In [70]:
normalized_dfs = []

# Itera sobre cada linha do DataFrame
for i, row in df_carroceu.iterrows():
    # Normaliza a coluna 'children.data' e adiciona uma coluna com o índice original
    normalized_df = pd.json_normalize(row['children.data'])
    normalized_df['index'] = i
    normalized_dfs.append(normalized_df)

# Concatena todos os DataFrames normalizados
normalized_df = pd.concat(normalized_dfs)

# Une o DataFrame normalizado ao DataFrame original
# Isso resulta em duplicação de linhas para cada item da lista dentro do 'children.data'
result_df = df_carroceu.drop(columns=['children.data']).merge(normalized_df, left_index=True, right_on='index')

# Remova a coluna 'index' se não for mais necessária
dataframe_carrocel = result_df.drop(columns=['index'])

In [71]:
dataframe_carrocel.head()

,id_account,id_midia,media_type_x,media_type_y,media_url,id
0,17841417530400616,18014406560219838,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,17872413915075936
1,17841417530400616,18014406560219838,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,18035366650865659
0,17841417530400616,18012911482864852,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,17953216079703099
1,17841417530400616,18012911482864852,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,17933873435780382
2,17841417530400616,18012911482864852,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,18026180065749737


## Tb Stories (criar uma chave unica, baseada na conta e data da extração)

In [128]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStories.json'
data_stories_sem_insights = pd.read_json(file_path)
data_stories_sem_insights.head()

,id,username,stories
0,17841425444516188,bodemeier.digital,NaN
1,17841417530400616,gabgalani,"{'data': [{'comments_count': 0, 'like_count': ..."


In [129]:
data_stories_json = pd.json_normalize(data_stories_sem_insights['stories'])
# data_stories_json = pd.json_normalize(data_stories_json['data'])
data_stories_json = pd.concat([data_stories_sem_insights, data_stories_json], axis=1)
data_stories_json = data_stories_json[['id', 'data']]
data_stories_normalizado = data_stories_json.dropna()

data_stories_normalizado.head()

,id,data
1,17841417530400616,"[{'comments_count': 0, 'like_count': 0, 'media..."


In [130]:
normalized_dfs = []

# Itera sobre cada linha do DataFrame
for i, row in data_stories_normalizado.iterrows():
    # Normaliza a coluna 'children.data' e adiciona uma coluna com o índice original
    normalized_df = pd.json_normalize(row['data'])
    normalized_df['index'] = i
    normalized_dfs.append(normalized_df)

# Concatena todos os DataFrames normalizados
normalized_df = pd.concat(normalized_dfs)

# Une o DataFrame normalizado ao DataFrame original
# Isso resulta em duplicação de linhas para cada item da lista dentro do 'children.data'
result_df = data_stories_normalizado.drop(columns=['data']).merge(normalized_df, left_index=True, right_on='index')

# Remova a coluna 'index' se não for mais necessária
data_stories_normalizado = result_df.drop(columns=['index'])
data_stories_normalizado.rename(columns={'id_x': 'id_account', 'id_y': 'id_story'}, inplace=True)


In [131]:
data_stories_normalizado.head()

,id_account,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,shortcode,id_story,owner.id,owner.username
0,17841417530400616,0,0,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,Eu te amo meu amor,2024-05-19T00:11:22+0000,https://instagram.com/stories/gabgalani/337102...,STORY,C7ISSBWOZA7,18035502889924169,17841417530400616,gabgalani


## Tb Account Day Metrics

In [90]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbDayAccounIgMetrics.json'
data_day_account = pd.read_json(file_path)
data_day_account.head()

,id,insights,username
0,17841425444516188,"{'data': [{'name': 'impressions', 'period': 'd...",bodemeier.digital
1,17841417530400616,"{'data': [{'name': 'impressions', 'period': 'd...",gabgalani


In [91]:
df_json = pd.json_normalize(data_day_account['insights'])
df_json = pd.concat([data_day_account, df_json], axis=1)
df_json.rename (columns={'id': 'id_account'}, inplace=True)
df_json = df_json[['id_account','data']]
df_json.head()

,id_account,data
0,17841425444516188,"[{'name': 'impressions', 'period': 'day', 'val..."
1,17841417530400616,"[{'name': 'impressions', 'period': 'day', 'val..."


In [95]:
df_primeira = pd.json_normalize(df_json['data'])
df_primeira.head()

,0,1,2,3,4,5,6,7,8
0,"{'name': 'impressions', 'period': 'day', 'valu...","{'name': 'reach', 'period': 'day', 'values': [...","{'name': 'follower_count', 'period': 'day', 'v...","{'name': 'email_contacts', 'period': 'day', 'v...","{'name': 'phone_call_clicks', 'period': 'day',...","{'name': 'text_message_clicks', 'period': 'day...","{'name': 'get_directions_clicks', 'period': 'd...","{'name': 'website_clicks', 'period': 'day', 'v...","{'name': 'profile_views', 'period': 'day', 'va..."
1,"{'name': 'impressions', 'period': 'day', 'valu...","{'name': 'reach', 'period': 'day', 'values': [...","{'name': 'follower_count', 'period': 'day', 'v...","{'name': 'email_contacts', 'period': 'day', 'v...","{'name': 'phone_call_clicks', 'period': 'day',...","{'name': 'text_message_clicks', 'period': 'day...","{'name': 'get_directions_clicks', 'period': 'd...","{'name': 'website_clicks', 'period': 'day', 'v...","{'name': 'profile_views', 'period': 'day', 'va..."


In [100]:
dfs = []
for c in df_primeira.columns: 
    json = pd.json_normalize(df_primeira[c])
    json_concat = pd.concat([df_json, json], axis=1)
    dfs.append(json_concat)

json_1_final = pd.concat(dfs)

In [195]:
json_1_final['chave'] = json_1_final['id_account'].astype(str) + json_1_final['name']
json_1_final.head()

,id_account,data,name,period,values,title,description,id,chave
0,17841425444516188,"[{'name': 'impressions', 'period': 'day', 'val...",impressions,day,"[{'value': 0, 'end_time': '2024-05-18T07:00:00...",Impressões,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/impressions/day,17841425444516188impressions
1,17841417530400616,"[{'name': 'impressions', 'period': 'day', 'val...",impressions,day,"[{'value': 59, 'end_time': '2024-05-18T07:00:0...",Impressões,Número total de vezes que os objetos de mídia ...,17841417530400616/insights/impressions/day,17841417530400616impressions
0,17841425444516188,"[{'name': 'impressions', 'period': 'day', 'val...",reach,day,"[{'value': 0, 'end_time': '2024-05-18T07:00:00...",Alcance,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/reach/day,17841425444516188reach
1,17841417530400616,"[{'name': 'impressions', 'period': 'day', 'val...",reach,day,"[{'value': 47, 'end_time': '2024-05-18T07:00:0...",Alcance,Número total de vezes que os objetos de mídia ...,17841417530400616/insights/reach/day,17841417530400616reach
0,17841425444516188,"[{'name': 'impressions', 'period': 'day', 'val...",follower_count,day,"[{'value': 0, 'end_time': '2024-05-18T07:00:00...",Número de seguidores,Número total de contas únicas que seguem este ...,17841425444516188/insights/follower_count/day,17841425444516188follower_count


In [170]:
teste_1 = json_1_final[['id_account', 'name', 'values']]

In [193]:
df_expanded = pd.concat([teste_1.drop(columns='values').reset_index(drop=True), pd.json_normalize(teste_1['values'])], axis=1)
df_expanded.head()

,id_account,name,0,1
0,17841425444516188,impressions,"{'value': 0, 'end_time': '2024-05-18T07:00:00+...","{'value': 0, 'end_time': '2024-05-19T07:00:00+..."
1,17841417530400616,impressions,"{'value': 59, 'end_time': '2024-05-18T07:00:00...","{'value': 31, 'end_time': '2024-05-19T07:00:00..."
2,17841425444516188,reach,"{'value': 0, 'end_time': '2024-05-18T07:00:00+...","{'value': 0, 'end_time': '2024-05-19T07:00:00+..."
3,17841417530400616,reach,"{'value': 47, 'end_time': '2024-05-18T07:00:00...","{'value': 26, 'end_time': '2024-05-19T07:00:00..."
4,17841425444516188,follower_count,"{'value': 0, 'end_time': '2024-05-18T07:00:00+...","{'value': 0, 'end_time': '2024-05-19T07:00:00+..."


In [197]:
json = df_expanded[[0,1]]
dfs = []
for i in json.columns: 
    columns = pd.json_normalize(json[i])
    if i == 0:
        columns.rename(columns={'value': 'last_day', 'end_time': 'last_end_time'}, inplace=True)
    if i == 1:
        columns.rename(columns={'value': 'actual_day', 'end_time': 'actual_end_time'}, inplace=True)
    col_concat = pd.concat([df_expanded, columns], axis=1)
    dfs.append(col_concat)

teste_final = pd.concat(dfs, axis=1)

In [201]:
select = ['id_account', 'name', 'last_day', 'last_end_time', 'actual_day', 'actual_end_time']
df_ = teste_final[select]
df_sem_duplicatas = df_.loc[:,~df_.columns.duplicated(keep='first')]
df_sem_duplicatas.head()

,id_account,name,last_day,last_end_time,actual_day,actual_end_time
0,17841425444516188,impressions,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000
1,17841417530400616,impressions,59,2024-05-18T07:00:00+0000,31,2024-05-19T07:00:00+0000
2,17841425444516188,reach,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000
3,17841417530400616,reach,47,2024-05-18T07:00:00+0000,26,2024-05-19T07:00:00+0000
4,17841425444516188,follower_count,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000


In [202]:
df_sem_duplicatas['chave'] = df_sem_duplicatas['id_account'].astype(str) + df_sem_duplicatas['name'] 
df_sem_duplicatas.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_13552\861708013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem_duplicatas['chave'] = df_sem_duplicatas['id_account'].astype(str) + df_sem_duplicatas['name']


,id_account,name,last_day,last_end_time,actual_day,actual_end_time,chave
0,17841425444516188,impressions,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000,17841425444516188impressions
1,17841417530400616,impressions,59,2024-05-18T07:00:00+0000,31,2024-05-19T07:00:00+0000,17841417530400616impressions
2,17841425444516188,reach,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000,17841425444516188reach
3,17841417530400616,reach,47,2024-05-18T07:00:00+0000,26,2024-05-19T07:00:00+0000,17841417530400616reach
4,17841425444516188,follower_count,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000,17841425444516188follower_count


In [205]:
resultado = json_1_final.merge(df_sem_duplicatas, left_on='chave', right_on='chave')

Acredito que manterei dessa forma

In [208]:
select = ['id_account_x', 'name_x', 'period', 'title', 'id', 'chave', 'last_day', 'last_end_time', 'actual_day', 'actual_end_time']
resultado = resultado[select]
resultado.head()

,id_account_x,name_x,period,title,id,chave,last_day,last_end_time,actual_day,actual_end_time
0,17841425444516188,impressions,day,Impressões,17841425444516188/insights/impressions/day,17841425444516188impressions,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000
1,17841417530400616,impressions,day,Impressões,17841417530400616/insights/impressions/day,17841417530400616impressions,59,2024-05-18T07:00:00+0000,31,2024-05-19T07:00:00+0000
2,17841425444516188,reach,day,Alcance,17841425444516188/insights/reach/day,17841425444516188reach,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000
3,17841417530400616,reach,day,Alcance,17841417530400616/insights/reach/day,17841417530400616reach,47,2024-05-18T07:00:00+0000,26,2024-05-19T07:00:00+0000
4,17841425444516188,follower_count,day,Número de seguidores,17841425444516188/insights/follower_count/day,17841425444516188follower_count,0,2024-05-18T07:00:00+0000,0,2024-05-19T07:00:00+0000


In [220]:
pivot_df = resultado.pivot_table(index=['id_account_x', 'period'], 
                          columns='name_x', 
                          values=['last_day', 'last_end_time', 'actual_day', 'actual_end_time'],
                          aggfunc='first')

# Flatten o MultiIndex das colunas
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# Resetando o índice
pivot_df.reset_index(inplace=True)

In [223]:
select = ['id_account_x', 'period', 
 'actual_day_email_contacts', 'last_day_email_contacts', 
 'actual_day_follower_count', 'last_day_follower_count',
 'actual_day_get_directions_clicks', 'last_day_get_directions_clicks',
 'actual_day_impressions',  'last_day_impressions',
'actual_day_phone_call_clicks', 'last_day_phone_call_clicks',
'actual_day_profile_views', 'last_day_profile_views',
'actual_day_reach', 'last_day_reach',
'actual_day_text_message_clicks',  'last_day_text_message_clicks',
'actual_day_website_clicks',   'last_day_website_clicks',
'actual_end_time_email_contacts','last_end_time_email_contacts',
]
pivot_df = pivot_df[select]

In [224]:
pivot_df.head()

,id_account_x,period,actual_day_email_contacts,last_day_email_contacts,actual_day_follower_count,last_day_follower_count,actual_day_get_directions_clicks,last_day_get_directions_clicks,actual_day_impressions,last_day_impressions,...,actual_day_profile_views,last_day_profile_views,actual_day_reach,last_day_reach,actual_day_text_message_clicks,last_day_text_message_clicks,actual_day_website_clicks,last_day_website_clicks,actual_end_time_email_contacts,last_end_time_email_contacts
0,17841417530400616,day,0,0,0,0,0,0,31,59,...,2,1,26,47,0,0,0,0,2024-05-19T07:00:00+0000,2024-05-18T07:00:00+0000
1,17841425444516188,day,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-19T07:00:00+0000,2024-05-18T07:00:00+0000


# Tb Account Insights lifetime

In [351]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbLifetimeAccounIgMetrics.json'
data_lifetime_account = pd.read_json(file_path)
data_lifetime_account.rename(columns={'id': 'id_account'}, inplace=True)
data_lifetime_account.head()

,id_account,insights,username
0,17841425444516188,"{'data': [{'name': 'audience_gender_age', 'per...",bodemeier.digital
1,17841417530400616,"{'data': [{'name': 'audience_gender_age', 'per...",gabgalani


In [352]:
json_data = pd.concat([data_lifetime_account, pd.json_normalize(pd.json_normalize(data_lifetime_account['insights'])['data'])], axis=1)
json_data.drop(columns='insights', inplace=True)

In [416]:
dataframes = []
for i in json_data.columns: 
    json_1 = pd.json_normalize(json_data[i])
    json_2 = pd.concat([json_data, json_1], axis=1)
    json_2.rename(columns={'values': f'values.{i}', 'title': f'title.{i}', 'description': f'description.{i}', 'name': f'name.{i}'}, inplace=True)
    dataframes.append(json_2)

df = pd.concat(dataframes, axis=1)
df_sem_duplicatas = df.loc[:, ~df.columns.duplicated()]
df_sem_duplicatas.drop(columns=[0,1,2,3,4, 'id'], inplace=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_13552\3856274431.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sem_duplicatas.drop(columns=[0,1,2,3,4, 'id'], inplace=True)


In [403]:
df_sem_duplicatas.head()

,id_account,username,name.0,period,values.0,title.0,description.0,name.1,values.1,title.1,description.1,name.2,values.2,title.2,description.2,name.3,values.3,title.3,description.3,name.4,values.4,title.4,description.4
0,17841425444516188,bodemeier.digital,audience_gender_age,lifetime,"[{'value': {'F.13-17': 3, 'F.18-24': 144, 'F.2...",Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,audience_locale,"[{'value': {'el_GR': 1, 'ru_RU': 5, 'it_IT': 5...",Localização,Localidades por códigos de país dos seguidores...,audience_country,"[{'value': {'DE': 1, 'BD': 1, 'RU': 10, 'PT': ...",País do público,Países dos seguidores deste perfil,audience_city,"[{'value': {'São Paulo, São Paulo (state)': 18...",Cidade do público,Cidades dos seguidores deste perfil,online_followers,"[{'value': {}, 'end_time': '2024-05-20T07:00:0...",Seguidores online,Número total de seguidores deste perfil que es...
1,17841417530400616,gabgalani,audience_gender_age,lifetime,"[{'value': {'F.18-24': 13, 'F.25-34': 41, 'F.3...",Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,audience_locale,"[{'value': {'es_LA': 1, 'en_US': 10, 'pt_BR': ...",Localização,Localidades por códigos de país dos seguidores...,audience_country,"[{'value': {'BR': 209, 'HK': 1, 'MY': 1, 'EC':...",País do público,Países dos seguidores deste perfil,audience_city,"[{'value': {'Planaltina, Goiás': 1, 'Ribeirão ...",Cidade do público,Cidades dos seguidores deste perfil,online_followers,"[{'value': {}, 'end_time': '2024-05-20T07:00:0...",Seguidores online,Número total de seguidores deste perfil que es...


## Tb faixa_etaria_genero

In [405]:
value_0_select = ['id_account', 'name.0', 'period', 'values.0', 'title.0','description.0']
value_0 = df_sem_duplicatas[value_0_select]
normalized_values_0 = pd.json_normalize(pd.json_normalize(value_0['values.0'])[0])
normalized_values_0 = pd.concat([value_0, normalized_values_0], axis=1)
normalized_values_0.drop(columns='values.0',inplace=True)
df_melted = pd.melt(normalized_values_0, 
                    id_vars=['id_account', 'name.0', 'period', 'title.0', 'description.0'], 
                    var_name='faixa_etaria_genero', 
                    value_name='valor')
df_melted.head()

,id_account,name.0,period,title.0,description.0,faixa_etaria_genero,valor
0,17841425444516188,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,value.F.13-17,3.0
1,17841417530400616,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,value.F.13-17,NaN
2,17841425444516188,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,value.F.18-24,144.0
3,17841417530400616,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,value.F.18-24,13.0
4,17841425444516188,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,value.F.25-34,308.0


## Tb localizacao_pais

In [414]:
value_1_select = ['id_account', 'name.1', 'period', 'values.1', 'title.1','description.1']
value_1 = df_sem_duplicatas[value_1_select]
normalized_values_1 = pd.json_normalize(pd.json_normalize(value_1['values.1'])[0])
normalized_values_1 = pd.concat([value_1, normalized_values_1], axis=1)
normalized_values_1.drop(columns='values.1',inplace=True)

df_melted_1 = pd.melt(normalized_values_1, 
                    id_vars=['id_account', 'name.1', 'period', 'title.1', 'description.1'], 
                    var_name='localizacao_pais', 
                    value_name='valor')
df_melted_1.head()

,id_account,name.1,period,title.1,description.1,localizacao_pais,valor
0,17841425444516188,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,value.el_GR,1.0
1,17841417530400616,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,value.el_GR,NaN
2,17841425444516188,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,value.ru_RU,5.0
3,17841417530400616,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,value.ru_RU,NaN
4,17841425444516188,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,value.it_IT,5.0


## tb Pais_do_publico

In [420]:
value_2_select = ['id_account', 'name.2', 'period', 'values.2', 'title.2','description.2']
value_2 = df_sem_duplicatas[value_2_select]
normalized_values_2 = pd.json_normalize(pd.json_normalize(value_2['values.2'])[0])
normalized_values_2 = pd.concat([value_2, normalized_values_2], axis=1)
normalized_values_2.drop(columns='values.2',inplace=True)
df_melted_2 = pd.melt(normalized_values_2, 
                    id_vars=['id_account', 'name.2', 'period', 'title.2', 'description.2'], 
                    var_name='Pais_do_publico', 
                    value_name='valor')
df_melted_2.head()

,id_account,name.2,period,title.2,description.2,Pais_do_publico,valor
0,17841425444516188,audience_country,lifetime,País do público,Países dos seguidores deste perfil,value.DE,1.0
1,17841417530400616,audience_country,lifetime,País do público,Países dos seguidores deste perfil,value.DE,NaN
2,17841425444516188,audience_country,lifetime,País do público,Países dos seguidores deste perfil,value.BD,1.0
3,17841417530400616,audience_country,lifetime,País do público,Países dos seguidores deste perfil,value.BD,NaN
4,17841425444516188,audience_country,lifetime,País do público,Países dos seguidores deste perfil,value.RU,10.0


## Cidade_dos_seguidores

In [423]:
value_3_select = ['id_account', 'name.3', 'period', 'values.3', 'title.3','description.3']
value_3 = df_sem_duplicatas[value_3_select]
normalized_values_3 = pd.json_normalize(pd.json_normalize(value_3['values.3'])[0])
normalized_values_3 = pd.concat([value_3, normalized_values_3], axis=1)
normalized_values_3.drop(columns='values.3',inplace=True)
df_melted_3 = pd.melt(normalized_values_3, 
                    id_vars=['id_account', 'name.3', 'period', 'title.3', 'description.3'], 
                    var_name='Cidade_dos_seguidores', 
                    value_name='valor')
df_melted_3.head()


,id_account,name.3,period,title.3,description.3,Cidade_dos_seguidores,valor
0,17841425444516188,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,"value.São Paulo, São Paulo (state)",180.0
1,17841417530400616,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,"value.São Paulo, São Paulo (state)",107.0
2,17841425444516188,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,"value.Rio de Janeiro, Rio de Janeiro (state)",24.0
3,17841417530400616,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,"value.Rio de Janeiro, Rio de Janeiro (state)",7.0
4,17841425444516188,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,"value.Yuma, Arizona",20.0


## Seguidores_online

In [425]:
value_4_select = ['id_account', 'name.4', 'period', 'values.4', 'title.4','description.4']
value_4 = df_sem_duplicatas[value_4_select]
normalized_values_4 = pd.json_normalize(pd.json_normalize(value_4['values.4'])[0])
normalized_values_4 = pd.concat([value_4, normalized_values_4], axis=1)
normalized_values_4.drop(columns='values.4',inplace=True)
df_melted_4 = pd.melt(normalized_values_4, 
                    id_vars=['id_account', 'name.4', 'period', 'title.4', 'description.4'], 
                    var_name='Seguidores_online', 
                    value_name='valor')
df_melted_4.head()

,id_account,name.4,period,title.4,description.4,Seguidores_online,valor
0,17841425444516188,online_followers,lifetime,Seguidores online,Número total de seguidores deste perfil que es...,end_time,2024-05-20T07:00:00+0000
1,17841417530400616,online_followers,lifetime,Seguidores online,Número total de seguidores deste perfil que es...,end_time,2024-05-20T07:00:00+0000


# Tb Stories insights

In [524]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStoriesInsightsLifetime.json'
data_lifetime_story = pd.read_json(file_path)
data_lifetime_story.dropna(subset=['stories'], inplace=True)
data_lifetime_story.rename(columns={'id': 'id_account'}, inplace=True)
data_lifetime_story.head()

,id_account,username,stories
1,17841417530400616,gabgalani,"{'data': [{'id': '17964631463739660', 'insight..."


In [525]:
stories_json = pd.concat([data_lifetime_story.reset_index(drop=True), pd.json_normalize(pd.json_normalize(data_lifetime_story['stories'])['data']).reset_index(drop=True)], axis=1)
stories_json.drop(columns='stories', inplace=True)
stories_json.head()

,id_account,username,0,1
0,17841417530400616,gabgalani,"{'id': '17964631463739660', 'insights.data': [...","{'id': '17995934558433765', 'insights.data': [..."


In [540]:
dfs_data = []
for i in stories_json.columns:
    data_json = pd.json_normalize(stories_json[i])
    dfs_data.append(data_json)

df_final_data = pd.concat(dfs_data)
df_final_data.dropna(inplace=True)
df_final_data.head()

,id,insights.data
0,17964631463739660,"[{'name': 'exits', 'period': 'lifetime', 'valu..."
0,17995934558433765,"[{'name': 'exits', 'period': 'lifetime', 'valu..."


In [546]:
df_exploded = df_final_data.explode('insights.data').reset_index(drop=True)
df_normalized = pd.json_normalize(df_exploded['insights.data'])
df_final = pd.concat([df_exploded[['id']].reset_index(drop=True), df_normalized.reset_index(drop=True)], axis=1)
df_values_normalized = pd.json_normalize(df_final['values'].explode())
df_final = pd.concat([df_final.drop(columns=['values']), df_values_normalized], axis=1)
df_final.head(20)

,id,name,period,title,description,id,value
0,17964631463739660,exits,lifetime,Saídas,Número de vezes que alguém saiu do story,17964631463739660/insights/exits/lifetime,9
1,17964631463739660,replies,lifetime,Respostas,Número total de respostas ao story,17964631463739660/insights/replies/lifetime,0
2,17964631463739660,taps_forward,lifetime,Toques para avançar,Número total de toques para ver a foto ou víde...,17964631463739660/insights/taps_forward/lifetime,30
3,17964631463739660,taps_back,lifetime,Toques para voltar,Número total de toques para ver a foto ou o ví...,17964631463739660/insights/taps_back/lifetime,2
4,17964631463739660,impressions,lifetime,Impressões,O número de vezes que seu story foi exibido na...,17964631463739660/insights/impressions/lifetime,31
5,17964631463739660,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este story...,17964631463739660/insights/reach/lifetime,30
6,17995934558433765,exits,lifetime,Saídas,Número de vezes que alguém saiu do story,17995934558433765/insights/exits/lifetime,7
7,17995934558433765,replies,lifetime,Respostas,Número total de respostas ao story,17995934558433765/insights/replies/lifetime,0
8,17995934558433765,taps_forward,lifetime,Toques para avançar,Número total de toques para ver a foto ou víde...,17995934558433765/insights/taps_forward/lifetime,32
9,17995934558433765,taps_back,lifetime,Toques para voltar,Número total de toques para ver a foto ou o ví...,17995934558433765/insights/taps_back/lifetime,0


# Tb Midias Insights Lifetime

In [548]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMediaInsightsLifetime.json'
data_lifetime_midia = pd.read_json(file_path)
data_lifetime_midia.dropna(subset=['media'], inplace=True)
data_lifetime_midia.rename(columns={'id': 'id_account'}, inplace=True)
data_lifetime_midia.head()

,id_account,media,username
0,17841425444516188,"{'data': [{'id': '18005273918369940', 'insight...",bodemeier.digital


In [550]:
stories_json = pd.concat([data_lifetime_midia.reset_index(drop=True), pd.json_normalize(pd.json_normalize(data_lifetime_midia['media'])['data']).reset_index(drop=True)], axis=1)
stories_json.drop(columns='media', inplace=True)
stories_json.head()

,id_account,username,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,17841425444516188,bodemeier.digital,"{'id': '18005273918369940', 'insights.data': [...","{'id': '17846370906157882', 'insights.data': [...","{'id': '18002752616172667', 'insights.data': [...","{'id': '17971594208670851', 'insights.data': [...","{'id': '18062694892504235', 'insights.data': [...","{'id': '17870243565059503', 'insights.data': [...","{'id': '17897388878957061', 'insights.data': [...","{'id': '18214911715272967', 'insights.data': [...","{'id': '17905338809918494', 'insights.data': [...","{'id': '17912491130893668', 'insights.data': [...","{'id': '18020750125974446', 'insights.data': [...","{'id': '17869447275058315', 'insights.data': [...","{'id': '18030343753699664', 'insights.data': [...","{'id': '18012618164496765', 'insights.data': [...","{'id': '18020150257990451', 'insights.data': [...","{'id': '18024592819898084', 'insights.data': [...","{'id': '17997324461240753', 'insights.data': [...","{'id': '17982183917471535', 'insights.data': [...","{'id': '17873173155025720', 'insights.data': [...","{'id': '17892179540901363', 'insights.data': [...","{'id': '17953225589736051', 'insights.data': [...","{'id': '18039366277634967', 'insights.data': [...","{'id': '18034865689681990', 'insights.data': [...","{'id': '18091892248364995', 'insights.data': [...","{'id': '17950801427740207', 'insights.data': [..."


In [551]:
dfs_data = []
for i in stories_json.columns:
    data_json = pd.json_normalize(stories_json[i])
    dfs_data.append(data_json)

df_final_data = pd.concat(dfs_data)
df_final_data.dropna(inplace=True)
df_final_data.head()

,id,insights.data
0,18005273918369940,"[{'name': 'reach', 'period': 'lifetime', 'valu..."
0,17846370906157882,"[{'name': 'reach', 'period': 'lifetime', 'valu..."
0,18002752616172667,"[{'name': 'reach', 'period': 'lifetime', 'valu..."
0,17971594208670851,"[{'name': 'reach', 'period': 'lifetime', 'valu..."
0,18062694892504235,"[{'name': 'reach', 'period': 'lifetime', 'valu..."


In [552]:
df_exploded = df_final_data.explode('insights.data').reset_index(drop=True)
df_normalized = pd.json_normalize(df_exploded['insights.data'])
df_final = pd.concat([df_exploded[['id']].reset_index(drop=True), df_normalized.reset_index(drop=True)], axis=1)
df_values_normalized = pd.json_normalize(df_final['values'].explode())
df_final = pd.concat([df_final.drop(columns=['values']), df_values_normalized], axis=1)
df_final.head(20)

,id,name,period,title,description,id,value
0,18005273918369940,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18005273918369940/insights/reach/lifetime,145
1,18005273918369940,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,18005273918369940/insights/saved/lifetime,1
2,17846370906157882,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,17846370906157882/insights/reach/lifetime,98
3,17846370906157882,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,17846370906157882/insights/saved/lifetime,0
4,18002752616172667,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18002752616172667/insights/reach/lifetime,123
5,18002752616172667,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,18002752616172667/insights/saved/lifetime,0
6,17971594208670851,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,17971594208670851/insights/reach/lifetime,138
7,17971594208670851,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,17971594208670851/insights/saved/lifetime,0
8,18062694892504235,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18062694892504235/insights/reach/lifetime,138
9,18062694892504235,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,18062694892504235/insights/saved/lifetime,0
